In [ ]:
from __future__ import print_function

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
import datetime as datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#%load_ext tensorboard

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)

In [ ]:
img_rows, img_cols = 28, 28

x_train_reshape = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test_reshape = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [ ]:
# input image dimensions
x_train_reshape = x_train_reshape.astype('float32')
x_test_reshape = x_test_reshape.astype('float32')

x_train_reshape /= 255
x_test_reshape /= 255

print('x_train shape:', x_train.shape)
print(x_train_reshape.shape[0], 'train samples')
print(x_test_reshape.shape[0], 'test samples')

In [ ]:
# convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test =to_categorical(y_test, 10)

In [ ]:
model = Sequential()

model.add(Conv2D(4, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,
                 kernel_initializer='he_normal',))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(loss="categorical_crossentropy",
              optimizer="Adam",
              metrics=['accuracy'])

In [ ]:
import os
#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = TensorBoard(logdir)

In [ ]:
history= model.fit(x_train_reshape, y_train,
          epochs=5,
          validation_data=(x_test_reshape, y_test),
          #callbacks=[tensorboard_callback]
                  )

# 8 minutes on GPU P1000
# 2 minutes on 2080TX

#val_accuracy: 0.9877

In [ ]:
score = model.evaluate(x_test_reshape, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Adam optimizer with he_normal:
#Test loss: 0.03299637805730106
#Test accuracy: 0.9916
    
# Adadelta with normal optimizer
#Test loss: 0.3832157447218895
#Test accuracy: 0.8958

# Added model.add(Conv2D(128, (3, 3), activation='relu')) and remove dropout
#Test loss: 0.04446429884471263
#Test accuracy: 0.9915

<h2> Visualize Layers</h2>

In [ ]:
model.summary()

<h3> Visualizing Filters </h3>

In [ ]:
import matplotlib.pyplot as plt

for layer in model.layers:
    if 'conv' not in layer.name:
        continue
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)

In [ ]:
# retrieve weights from the second hidden layer
filters, biases = model.layers[0].get_weights()

# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

In [ ]:
# plot first few filters
n_filters, ix = 4, 1
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(1):
        # specify subplot and turn of axis
        ax = plt.subplot(n_filters, 3, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        print(f[:, :, j])
        plt.imshow(f[:, :, j], cmap='gray')
        ix += 1
# show the figure
plt.show()

<h3> Visualizing Features Maps </h3>

In [ ]:
from tensorflow.keras.models import Model

# Extract an image and visualize its value

sample_mnsit = x_train[2]
plt.imshow(sample_mnsit, cmap='gray')

In [ ]:
# fresh copy of image
sample_mnsit = x_train[2]

# Reshape to fit classifier
sample_mnsit = sample_mnsit.reshape(1, img_rows, img_cols, 1)

#Convert to flaot32
sample_mnsit = sample_mnsit.astype('float32')

# Normalizr features
sample_mnsit /= 255

# Input layer number you want to extract the feature maps from
layer_no = 0

# Get the filters from the first layer
modified_model = Model(inputs=model.inputs, outputs=model.layers[layer_no].output)

# Apply filters to image to get activation function
feature_maps  = modified_model.predict(sample_mnsit)

# Plot graph of each feature map
nrows = int(feature_maps.shape[-1] / 2)
ncols = 2
fig, axs = plt.subplots(nrows,ncols, 
                        figsize=(20, 20), 
                        facecolor='w', 
                        edgecolor='k',
                       )

axs = axs.ravel()


for i in range(feature_maps.shape[-1]):

    axs[i].imshow(feature_maps[0, :, :, i], cmap='gray')

plt.tight_layout(True)
plt.show()

In [ ]:
import cv2, numpy as np
from sklearn.cluster import KMeans

# Get the top dominat colors by percentage
def get_cluser(image):
    reshape = image.reshape((image.shape[0] * image.shape[1], 1))
    cluster = KMeans(n_clusters=5).fit(reshape)
    
    return cluster, cluster.cluster_centers_


def visualize_colors(image):
    
    cluster, centroids = get_cluser(image)
    # Get the number of different clusters, create histogram, and normalize
    labels = np.arange(0, len(np.unique(cluster.labels_)) + 1)
    (hist, _) = np.histogram(cluster.labels_, bins = labels)
    hist = hist.astype("float")
    hist /= hist.sum()

    # Create frequency rect and iterate through each cluster's color and percentage
    colors = sorted([(percent, color) for (percent, color) in zip(hist, centroids)])
    
    return [percent*100 for (percent, color) in colors]
        


# Get the top 5 dominant color of the original image
original_image = visualize_colors(x_train[2])

# Get the top 5 dominant colors of the feature maps, subtract from original image
# Get mean value of the 5 differnce 

mean_value_diff = []
for x in range(feature_maps.shape[-1]):
    feature_map = visualize_colors(feature_maps[0, :, :, x])
    mean_value_diff.append(np.mean([np.abs(a_i - b_i) for a_i, b_i in zip(original_image, feature_map)]))
    
# values of sorted top feature maps
top_similair_features_values= sorted(mean_value_diff)

# index of sorted top feature maps with similiar value
top_similair_features_index = sorted(range(len(mean_value_diff)), key=lambda k: mean_value_diff[k])


# Plot top similiar features with original image
fig, axs = plt.subplots(nrows,ncols, 
                        figsize=(20, 20), 
                        facecolor='w', 
                        edgecolor='k',
                       )

axs = axs.ravel()



for i in range(feature_maps.shape[-1]):

    axs[i].imshow(feature_maps[0, :, :, top_similair_features_index[i]], cmap='gray')
    axs[i].text(0.5,-0.1, top_similair_features_values[i], size=12, ha="center", 
             transform=axs[i].transAxes)
plt.tight_layout(True)
plt.show()


<h2> Archived </h2>

In [ ]:
# Check difference between methods

model_1 = Sequential()

model_1.add(Conv2D(4, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,
                 kernel_initializer='he_normal',))
model_1.add(Flatten())
model_1.add(Dense(10, activation='softmax'))

model_1.compile(loss="categorical_crossentropy",
              optimizer="Adam",
              metrics=['accuracy'])

history= model_1.fit(x_train_reshape, y_train,
          epochs=2,
          verbose=1,
          validation_data=(x_test_reshape, y_test)
                  )

In [ ]:
# use the following to get the parameters of the weights
model_1.summary()
model_1.layers[0].get_weights()
#model_1.get_weights()[0]

In [ ]:
weights, biases = model_1.layers[0].get_weights()
column_one = [weights[0][x][0][0] for x in range(3)]
column_two = [weights[1][x][0][0] for x in range(3)]
column_three = [weights[2][x][0][0] for x in range(3)]

print(column_one, column_two, column_three)

In [ ]:
w_2,b_2 = model_1.layers[2].get_weights()
w_2.shape

In [ ]:
inp = Input((28,28,1))

# Conv layer
w,b = model_1.layers[0].get_weights()
w = np.delete(w, [0], -1)
b = np.delete(b, [0], 0)

# FC layer
w_2,b_2 = model_1.layers[2].get_weights()
w_2 = w[673:,:]

# Conv2D neural network
new_c = Conv2D(3, kernel_size=(3, 3),
               activation='relu',
               kernel_initializer='he_normal',
               trainable=False)
f = Flatten()
d = Dense(10, activation='softmax')

x = new_c(inp)
x = f(x)
out = d(x) # -----> error!
new_model= Model(inp, out)


print(new_model.summary())

In [ ]:
#What is happening here?
#new_model.predict(sample_mnsit)
score = model.evaluate(x_test_reshape, y_test, verbose=0)
print(score)